## Instalar dependencias

Nesta parte foi um pouco complicado, pois havia algumas incompatibilidades que não estavam me permitando instalar da forma correta o basedosdados. Então, o que eu fiz foi usar o python na versão 3.10 e instalar o basedosdados na versão 2.0.0b15. Após isso, eu voltei para a versão 3.12 do python e funcionou.

In [ ]:
pip install basedosdados

## Importar Bibliotecas necessárias

In [ ]:
import basedosdados as bd
import pandas as pd
from IPython.display import display

## Questão 1
Quantos chamados foram abertos no dia 01/04/2023?

In [ ]:
q1 = """
SELECT COUNT(*) AS chamados_abertos
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) = '2023-04-01'
"""
que1 = bd.read_sql(q1, billing_project_id="sinuous-gist-433018-f3")
que1.head()

Resposta: 1756 chamados abertos no dia 01-04-2023

## Questão 2:
Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

In [ ]:
q2 = """
SELECT tipo, COUNT(*) AS qtd_chamados
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) = '2023-04-01'
GROUP BY tipo
ORDER BY qtd_chamados DESC
LIMIT 1
"""
que2 = bd.read_sql(q2, billing_project_id="sinuous-gist-433018-f3")
que2.head()

Resposta: Estacionamento irregular, com 366 chamados

## Questão 3:
Questão 3: Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [ ]:
q3 = """
SELECT id_bairro, COUNT(*) AS qtd_chamados_bairro
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) = '2023-04-01'
GROUP BY id_bairro
ORDER BY qtd_chamados_bairro DESC
LIMIT 4
"""

q3_1 = """
SELECT *
FROM `datario.dados_mestres.bairro`
WHERE id_bairro IN ('144', '33','128')
"""
que3_1 = bd.read_sql(q3_1, billing_project_id="sinuous-gist-433018-f3")
que3 = bd.read_sql(q3, billing_project_id="sinuous-gist-433018-f3")

display(que3.head())
display(que3_1.head())

Resposta:
Nesta questão, eu filtrei os 3 bairros com mais chamados, porém o 3º bairro não havia<br>
id de identificação do bairro, dessa forma eu desconsiderei o 3º colocado e fui para o 4º bairro com mais<br>
ocorrências.<br>
    1º : Campo Grande, ID 144 - Chamados : 113<br>
    2º : Tijuca, ID 33 - Chamados : 89<br>
    3º : Barra da Tijuca, ID 128 - Chamados : 59<br>
    3º : NULL - Chamados : 73 ( Provavelmente, são os chamados que não tem ligação com um lugar fisico.)<br>

## Questão 4:
Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [ ]:
q4 = """
SELECT b.subprefeitura, COUNT(c.id_bairro) AS qtd_chamados_subprefeitura
FROM datario.adm_central_atendimento_1746.chamado c
JOIN datario.dados_mestres.bairro b
ON c.id_bairro = b.id_bairro
WHERE DATE(c.data_inicio) = '2023-04-01'
GROUP BY b.subprefeitura
ORDER BY qtd_chamados_subprefeitura DESC
LIMIT 1;
"""
que4 = bd.read_sql(q4, billing_project_id="sinuous-gist-433018-f3")
que4.head()

Resposta: Subprefeitura Zona Norte, com 510 chamados

## Questão 5:
Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros?Se sim, por que isso acontece?

In [ ]:
q5 = """
SELECT tipo,subtipo, data_inicio,id_bairro,
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) = '2023-04-01' 
  AND id_bairro IS NULL
LIMIT 100
"""
que5 = bd.read_sql(q5, billing_project_id="sinuous-gist-433018-f3")
que5.head(100)

Resposta: 
Sim, existe. Isso ocorre nas chamadas que são relacionadas ao transporte público, atendimento ao publico, defesa do consumidor,<br>
alvára, que não tem ligação com um lugar fisico, que no nosso caso de interesse seria um bairro.

## Questão 6:
Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [ ]:
q6 = """
SELECT subtipo, COUNT(*) AS qtd_chamados_pertubacao
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
  AND subtipo = 'Perturbação do sossego'
GROUP BY subtipo
ORDER BY qtd_chamados_pertubacao DESC
"""
que6 = bd.read_sql(q6, billing_project_id="sinuous-gist-433018-f3")
que6.head()

Resposta: Foram realizados 42830 chamados neste intervalo de datas com o subtipo Perturbação do sossego.

## Questão 7:
Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [ ]:
q7 = """
SELECT c.id_chamado, c.tipo, c.subtipo, c.data_inicio, e.evento AS Evento_no_Dia,
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
ON DATE(c.data_inicio) = DATE(e.data_inicial)
WHERE c.subtipo = 'Perturbação do sossego'
  AND e.taxa_ocupacao IN (0.9554, 0.9251, 0.9451, 0.8184)
ORDER BY e.data_inicial ASC
"""
que7 = bd.read_sql(q7, billing_project_id="sinuous-gist-433018-f3")
que7.head()

Resposta:

Utilizei o taxa_ocupacao como paramêtro para localizar o eventos<br>
pois existia eventos com o mesmo nome e isso iria me trazer problemas, por isso<br>
decidi utilizar o taxa_ocupação.<br>

      Não deixei todas as ocorrências visiveis, pois são muitas
      e ficaria "pesado" para o IDE.

## Questão 8:
Quantos chamados desse subtipo foram abertos em cada evento?

In [ ]:
q8 = """
SELECT c.evento, c.taxa_ocupacao,c.data_inicial, c.data_final, COUNT(*) AS qtd_chamados_eventos
FROM datario.adm_central_atendimento_1746.chamado b
JOIN datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos c
ON DATE(b.data_inicio) BETWEEN DATE(c.data_inicial) AND DATE(c.data_final)
WHERE b.subtipo = 'Perturbação do sossego'
  AND c.taxa_ocupacao IN (0.9554, 0.9251, 0.9451, 0.8184)
GROUP BY c.evento, c.taxa_ocupacao,c.data_inicial, c.data_final
ORDER BY qtd_chamados_eventos DESC
"""
que8 = bd.read_sql(q8, billing_project_id="sinuous-gist-433018-f3")
que8.head()

Resposta:<br>
        Rock In Rio ( 08-09-2022 ) : 468 Chamados<br>
        Rock In Rio ( 02-09-2022 ) : 366 Chamados<br>
        Carnaval ( 18-02-2023 ) : 241 Chamados<br>
        Reveillon ( 30-12-2022 ) : 139 Chamados<br>
<br>
Eu vi a necessidade de acrescentar a taxa_ocupacao, pelo fato que de há dois eventos
com o mesmo nome, Rock In Rio.

## Questão 9:
Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [ ]:
q9 = """
WITH contador AS (
  SELECT c.data_inicial ,c.evento,c.taxa_ocupacao ,DATE(b.data_inicio) AS dia, COUNT (*) AS total_chamados
  FROM `datario.adm_central_atendimento_1746.chamado` b
  JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` c
  ON DATE(b.data_inicio) BETWEEN DATE(c.data_inicial) AND DATE(c.data_final)
  WHERE b.subtipo = 'Perturbação do sossego'
  GROUP BY c.data_inicial, c.evento,c.taxa_ocupacao, dia
),

media_dias AS(
  SELECT data_inicial ,evento ,taxa_ocupacao, AVG(total_chamados) AS media_diaria
  FROM contador
  GROUP BY  data_inicial,evento, taxa_ocupacao
)

SELECT evento ,data_inicial, media_diaria,taxa_ocupacao
FROM media_dias
ORDER BY media_diaria DESC
LIMIT 4
"""
que9 = bd.read_sql(q9, billing_project_id="sinuous-gist-433018-f3")
que9.head()

Resposta: Aqui eu criei duas CTE's para auxiliar na legibilidade do código,<br>
então em contador eu faço a contagem em favor das condições e depois eu faço<br>
a média.<br>
<br>
Evento com maior média: Rock In Rio (02-09-2022) = 122,0 

## Questão 10:
Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos
    (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos
    desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

Pegando a Média Diaria Geral

In [ ]:
q10 = """
WITH contador AS (
  SELECT DATE(data_inicio) AS dia, COUNT(*) AS total_chamados_geral
  FROM `datario.adm_central_atendimento_1746.chamado`
  WHERE subtipo = 'Perturbação do sossego'
    AND DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
  GROUP BY dia
),

media_total AS (
  SELECT AVG(total_chamados_geral) AS media_diaria_geral
  FROM contador
)

SELECT media_diaria_geral
FROM media_total
ORDER BY media_diaria_geral DESC
"""
que10 = bd.read_sql(q10, billing_project_id="sinuous-gist-433018-f3")
que10.head()

Criar Coluna

In [ ]:
que10['tipo'] = 'Geral'
que9['tipo'] = que9['evento']

Merge dos DataFrames e organizando as colunas

In [ ]:
comparacao = pd.concat([que10, que9], ignore_index=True)
colunas = ['tipo', 'evento', 'data_inicial', 'media_diaria', 'media_diaria_geral', 'taxa_ocupacao']
comparacao = comparacao[colunas]

In [ ]:
comparacao.head()

Resposta: <br>Aqui, com o poder do python e pandas eu consegui realizar uma comparação melhor<br>
e vemos que a média diaria dos eventos Rock in Rio, são bem maiores que a média geral,<br>
mas é importante lembrar que a quantidade de dias é muito superior na média geral,<br>
é quase 2 anos de análise, então com certeza a média iria abaixar, mas é interessante ver<br>
como um evento grande como o Rock in Rio altera a dinâmica da cidade.